# Imports

In [49]:
import pandas as pd
import json
import validate
from general import many_to_many, many_cols
import ingest
import clean

# Pull Data into Pandas

In [50]:
data = ingest.main('inventory_correct_linking.xlsx')

In [51]:
excel_data = data['excel_data']
db = data['database']

# Cleaning

In [52]:
db = clean.remove_NaN_columns(db)

In [53]:
db = clean.strip_all_columns(db)

In [54]:
# change ignore_code field to be string
db['gcmd_phenomena']['ignore_code'] = db['gcmd_phenomena']['ignore_code'].apply(lambda x: str(x))

# Short Name Supplementation

In [55]:
# make unique shortname by combining the campaign name and the table sub short name

db['deployment']['short_name'] = db['deployment']['foreign-campaign-short_name']+'_'+db['deployment']['ignore_deployment_id']
db['iopse']['foreign-deployment-short_name']=db['iopse']['foreign-campaign-short_name']+'_'+db['iopse']['ignore_deployment']

db['collection_period'] = many_to_many(db, 'linking', 'table-instrument-short_name', keep_all=True)
db['collection_period']['short_name'] = db['collection_period']['foreign-campaign-short_name']+'_'+db['collection_period']['foreign-deployment-short_name']

# Find Matching Deployments for IOPSE

In [56]:
# original process before good shortname usage on the sheets


# db['iopse']['deployment_short_name'] = 'No Matches'

# for row in range(len(db['iopse'])):
#     iop_start = db['iopse'].iloc[row]['start_date']
#     iop_end = db['iopse'].iloc[row]['end_date']
#     iop_camp = db['iopse'].iloc[row]['foreign-campaign-short_name']
    
#     campaign_filter = db['deployment']['foreign-campaign-short_name'].apply(lambda short_name: short_name == iop_camp)
#     possible_campaigns = db['deployment'][campaign_filter]
    
#     start_filter = possible_campaigns['start_date'].apply(lambda dep_start: validate.vali_date(dep_start, iop_start))
#     end_filter = possible_campaigns['end_date'].apply(lambda dep_end: validate.vali_date(iop_end, dep_end))
    
#     matching_deployments = list(possible_campaigns[start_filter*end_filter]['short_name'])
#     if len(matching_deployments)>1:
#         print(f"error on {db['iopse'].iloc[row]}")
#     elif len(matching_deployments)==1:
#         matching_deployments = matching_deployments[0]
#     else:
#         matching_deployments = 'None Found'
#     db['iopse']['deployment_short_name'].iloc[row]=matching_deployments

In [57]:
# test for unexpected values in this column
assert set(db['iopse']['type']) == {'IOP', 'SE'}

In [58]:
# convert parent and short name to lower so they will match correctly
db['iopse']['short_name'] = db['iopse']['short_name'].apply(lambda x: x.lower())
db['iopse']['parent short_name'] = db['iopse']['parent short_name'].apply(lambda x: x.lower())

In [59]:
db['iop'] = db['iopse'][db['iopse']['type']=='IOP']
db['significant_event'] = db['iopse'][db['iopse']['type']=='SE']

# Many to Many Creation

In [60]:
# main_table_names = ['campaign', 'platform', 'instrument', 'deployment']

In [61]:
# for table in main_table_names:
#     print(table)
#     for column in [col for col in db[table].keys() if isinstance(col,str) and 'table' in col]:
#         name = column.split('-')[1]
#         new_table_name = f"{table}-to-{name}"
#         db[new_table_name]=many_to_many(db, table, column)
#         print(f'   {new_table_name} created')

# Campaign Filter

In [62]:
campaign_filter = [
    "ACES",
    "AirMOSS",
    "ARCTAS",
    "CARVE",
    "DC3",
    "GCPEx",
    "GOES-R PLT",
    "GRIP",
    "HS3",
    "OLYMPEX"
]

In [63]:
# campaigns
db['campaign'] = db['campaign'][db['campaign']['short_name'].apply(lambda x: x in campaign_filter)]
assert set(campaign_filter) == set(db['campaign']['short_name'])

In [64]:
# deployments
db['deployment']=db['deployment'][db['deployment']['foreign-campaign-short_name'].apply(lambda short: short in campaign_filter)]

In [65]:
# collection periods
db['collection_period']=db['collection_period'][db['collection_period']['foreign-campaign-short_name'].apply(lambda short: short in campaign_filter)]

In [66]:
# platforms
platform_filter = list(set(list(db['collection_period']['foreign-platform-short_name'])))
db['platform']=db['platform'][db['platform']['short_name'].apply(lambda short: short in platform_filter)]
print('\ncopy these platforms into a file for inventory folks\n')
[print(thing) for thing in platform_filter]


copy these platforms into a file for inventory folks

Citation
G-III
P-3
ALAR
DC-8
GH
CV-580
G-V
WB-57
B-200
Field_Site
C-23 Sherpa
ER-2
UND Citation II
Falcon
ASO
ALTUS II


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [67]:
instrument_filter = list(set(list(db['collection_period']['instrument'])))
db['instrument']=db['instrument'][db['instrument']['short_name'].apply(lambda short: short in instrument_filter)]
print('\ncopy these instruments into a file for inventory folks\n')
[print(thing) for thing in instrument_filter]


copy these instruments into a file for inventory folks

PIP probe
EFM
Accelerometer
Gerdien Probe
PALMS
PCAP
HD-SP2
Picarro
DOPS
FGM
FPDS
CSD CL
BBR
Nevzorov probe
CSI
FSSP
CAR
OAP-2G-P
HARP
PFP
HIWRAP
MMS
TOGA
CAPS
GCAS
AirMOSS
HAMSR
HVPS-3
UHSAS
RICE
TE49C
DIAL-HSRL
LASE
Gen-Chemistry
DIAL
FLIR
CAMS
CIMS
ATLAS
FEGS
EXRAD
SMPS
AVIRIS
SSFR
AMS
Spectrometer
IR-CO2
CIP
WS-CRDS
MSC
Aerolaser
Gen-AtmsState
GC-MS
OPC
HIRAD
PALS
King hot wire probe
DFGAS
PILS
CPL
PTR-MS
DLH
ATHOS
FTS
CRS
DAWN
HR-AMS
RSP
SR
WAS
DACOM
AVOCET
Slow Antenna
TDMA
ATSP
CIT-CIMS
CCP
EFCM
PSAP
NAWX radar
CNC
Canisters
CAFS
SP2
DASH
LARGE
CPSD
TD-LIF
2D-C/P
Dropsonde
APR-2
CDP
VCSEL
PI-Neph
WCN
AOP
PCASP
BAT probe
Aethalometer
SP-2
SPEC
GT-CIMS
AVAPS
S-HIS
3V-CPI
CPI
DADS
CCN
CLH
TDL
LIP
HSRL
Information Not Available
DV
ISAF
Nephelometer
CPC
SAGA
P-CIMS
CoSMIR
CI-ITMS


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

# Many to Many Creation

In [68]:
main_table_names = ['campaign', 'platform', 'instrument', 'deployment']

In [69]:
for table in main_table_names:
    print(table)
    for column in [col for col in db[table].keys() if isinstance(col,str) and 'table' in col]:
        name = column.split('-')[1]
        new_table_name = f"{table}-to-{name}"
        db[new_table_name]=many_to_many(db, table, column)
        print(f'   {new_table_name} created')

campaign
   campaign-to-focus_area created
   campaign-to-season created
   campaign-to-platform_type created
   campaign-to-gcmd_phenomena created
   campaign-to-repository created
   campaign-to-partner_org created
   campaign-to-gcmd_project created
platform
   platform-to-gcmd_platform created
instrument
   instrument-to-gcmd_instrument created
   instrument-to-instrument_type created
   instrument-to-measurement_keywords created
   instrument-to-geophysical_concept created
   instrument-to-repository created
   instrument-to-measurement_region created
deployment
   deployment-to-geographical_region created


# Validation

### Short Name Duplicates

In [70]:
for table_name in db.keys():
    if table_name == 'collection_period':
        # this is being skipped because it has been broken out by instrument
        # and therefore it has duplicate short_names
        continue
        
    if 'short_name' in db[table_name].keys():
        print(table_name)
        duplicates = validate.find_duplicates(db, table_name, 'short_name')

        print(f'    {duplicates}')

platform_type
    []
home_base
    []
repository
    []
focus_area
    []
season
    []
instrument_type
    []
measurement_region
    []
geographical_region
    []
geophysical_concept
    []
campaign
    []
platform
    []
instrument
    ['atlas']
deployment
    []
iopse
    []
gcmd_instrument
    ['atlas', 'informationnotavailable', 'wcr', nan, 'epic', 'opc', 'gnssreceiver', 'cris', 'aa', 'iris', 'particlespectrometers', 'ssies', 'icecube', 'aps']
gcmd_platform
    ['kingair', 'informationnotavailable', 'goes10', 'goes11', 'goes12', 'goes13', 'goes14', 'goes15', 'goes16', 'goes1', 'goes2', 'goes3', 'goes4', 'goes5', 'goes6', 'goes7', 'goes8', 'goes9', 'environmentalmodeling']
gcmd_project
    ['afsisclimate', 'camp', 'informationnotavailable', 'iodp', 'landsat7', 'mcmurdopredatorprey', 'notapplicable']
partner_org
    []
iop
    []
significant_event
    []


### Foriegn Key Links

### Campaign

In [71]:
errors = validate.foriegn_keys(db, 
                                data_table='campaign-to-gcmd_project', 
                                data_index='campaign', 
                                data_column='gcmd_project', 
                                foriegn_table='gcmd_project', 
                                foriegn_column='gcmd_uuid')
errors

,campaign,gcmd_project,suggestions


In [72]:
errors = validate.foriegn_keys(db, 
                                data_table='campaign-to-focus_area', 
                                data_index='campaign', 
                                data_column='focus_area', 
                                foriegn_table='focus_area', 
                                foriegn_column='short_name')
errors

,campaign,focus_area,suggestions


In [73]:
errors = validate.foriegn_keys(db, 
                                data_table='campaign-to-season', 
                                data_index='campaign', 
                                data_column='season', 
                                foriegn_table='season', 
                                foriegn_column='short_name')
errors

,campaign,season,suggestions


In [74]:
errors = validate.foriegn_keys(db, 
                                data_table='campaign-to-platform_type', 
                                data_index='campaign', 
                                data_column='platform_type', 
                                foriegn_table='platform_type', 
                                foriegn_column='short_name')
errors

,campaign,platform_type,suggestions


In [75]:
errors = validate.foriegn_keys(db, 
                                data_table='campaign-to-gcmd_phenomena', 
                                data_index='campaign', 
                                data_column='gcmd_phenomena', 
                                foriegn_table='gcmd_phenomena', 
                                foriegn_column='ignore_code')
errors

,campaign,gcmd_phenomena,suggestions


In [76]:
errors = validate.foriegn_keys(db, 
                                data_table='campaign-to-repository', 
                                data_index='campaign', 
                                data_column='repository', 
                                foriegn_table='repository', 
                                foriegn_column='short_name')
errors

,campaign,repository,suggestions


In [77]:
errors = validate.foriegn_keys(db, 
                                data_table='campaign-to-partner_org', 
                                data_index='campaign', 
                                data_column='partner_org', 
                                foriegn_table='partner_org', 
                                foriegn_column='short_name')
errors

,campaign,partner_org,suggestions


In [78]:
errors = validate.foriegn_keys(db, 
                                data_table='platform-to-gcmd_platform', 
                                data_index='platform', 
                                data_column='gcmd_platform', 
                                foriegn_table='gcmd_platform', 
                                foriegn_column='gcmd_uuid')
errors

,platform,gcmd_platform,suggestions


In [79]:
errors = validate.foriegn_keys(db, 
                                data_table='instrument-to-measurement_keywords', 
                                data_index='instrument', 
                                data_column='measurement_keywords', 
                                foriegn_table='gcmd_phenomena', 
                                foriegn_column='ignore_code')
errors

,instrument,measurement_keywords,suggestions


In [91]:
'6238f3e2-9a87-4e32-b866-c4a637094b51' == 
'6238fe2-9a87-4e32-b866-c4a637094b51'

False

In [80]:
errors = validate.foriegn_keys(db, 
                                data_table='instrument-to-gcmd_instrument', 
                                data_index='instrument', 
                                data_column='gcmd_instrument', 
                                foriegn_table='gcmd_instrument', 
                                foriegn_column='gcmd_uuid')
errors

,instrument,gcmd_instrument,suggestions
3,3V-CPI,NID,[]
7,AirMOSS,NID,[]
19,BAT probe,NID,[]
31,CIP,92f99316-b581-4adb-9980-aeb6bed64eee,[]
39,CPL,6238fe2-9a87-4e32-b866-c4a637094b51,[6238f3e2-9a87-4e32-b866-c4a637094b51]


In [81]:
errors = validate.foriegn_keys(db, 
                                data_table='instrument-to-instrument_type', 
                                data_index='instrument', 
                                data_column='instrument_type', 
                                foriegn_table='instrument_type', 
                                foriegn_column='short_name')
errors

,instrument,instrument_type,suggestions
13,AVAPS,Passive - Remote Sensing,[]
70,HIWRAP,Earth Remote Sensing - Active Remote Sensing,[]


In [82]:
errors = validate.foriegn_keys(db, 
                                data_table='instrument-to-geophysical_concept', 
                                data_index='instrument', 
                                data_column='geophysical_concept', 
                                foriegn_table='geophysical_concept', 
                                foriegn_column='short_name')
errors

,instrument,geophysical_concept,suggestions
1,2D-C/P,Precipitation,[Precipitation (Precip)]
2,3V-CPI,Precipitation,[Precipitation (Precip)]
7,AirMOSS,Terrestrial Hydrology,[Terrestrial Hydrology (TerrHydrol)]
10,APR-2,Precipitation,[Precipitation (Precip)]
55,EXRAD,Precipitation,[Precipitation (Precip)]
72,HIWRAP,Precipitation,[Precipitation (Precip)]
101,RICE,Cloud Properties,[]
102,RICE,Processes & Dynamics,[]
119,VCSEL,Atmospheric Chemicals & Trace Gasses,[Atmospheric Chemicals & Trace Gases]


In [83]:
errors = validate.foriegn_keys(db, 
                                data_table='instrument-to-repository', 
                                data_index='instrument', 
                                data_column='repository', 
                                foriegn_table='repository', 
                                foriegn_column='short_name')
errors

,instrument,repository,suggestions
21,BAT probe,ORNL,[]


In [84]:
errors = validate.foriegn_keys(db, 
                                data_table='instrument-to-measurement_region', 
                                data_index='instrument', 
                                data_column='measurement_region', 
                                foriegn_table='measurement_region', 
                                foriegn_column='short_name')
errors

,instrument,measurement_region,suggestions
1,3V-CPI,Troposphere,"[mid-troposphere, stratosphere]"
6,AirMOSS,subsurface,"[sea surface, subsurface - water, surface, sub..."
38,CRS,Troposphere,"[mid-troposphere, stratosphere]"
54,EXRAD,Troposphere,"[mid-troposphere, stratosphere]"
110,TD-LIF,troposphere,"[mid-troposphere, stratosphere]"


In [85]:
errors = validate.foriegn_keys(db, 
                                data_table='deployment-to-geographical_region', 
                                data_index='deployment', 
                                data_column='geographical_region', 
                                foriegn_table='geographical_region', 
                                foriegn_column='short_name')
errors

,deployment,geographical_region,suggestions


In [38]:
[key for key in db.keys() if 'deployment' in key]

['deployment', 'deployment-to-geographical_region']

In [39]:
errors = validate.foriegn_keys(db, 
                                data_table='deployment-to-platform', 
                                data_index='deployment', 
                                data_column='platform', 
                                foriegn_table='platform', 
                                foriegn_column='short_name')
print('\n\ndo I really need to validate this?')
errors

KeyError: 'deployment-to-platform'

In [40]:
errors = validate.foriegn_keys(db, 
                                data_table='deployment', 
                                data_index='short_name', 
                                data_column='foreign-campaign-short_name', 
                                foriegn_table='campaign', 
                                foriegn_column='short_name')
print('\n\n TODO this better once you have the data')
errors



 TODO this better once you have the data


6,short_name,foreign-campaign-short_name,suggestions


In [41]:
errors = validate.foriegn_keys(db, 
                                data_table='platform', 
                                data_index='short_name', 
                                data_column='foreign-platform_type-short_name', 
                                foriegn_table='platform_type', 
                                foriegn_column='short_name')
errors

2,short_name,foreign-platform_type-short_name,suggestions


In [42]:
errors = validate.foriegn_keys(db, 
                                data_table='platform-to-gcmd_platform', 
                                data_index='platform', 
                                data_column='gcmd_platform', 
                                foriegn_table='gcmd_platform', 
                                foriegn_column='gcmd_uuid')
errors

,platform,gcmd_platform,suggestions


In [43]:
# TODO: Handle case

errors = validate.foriegn_keys(db, 
                                data_table='iopse', 
                                data_index='short_name', 
                                data_column='foreign-deployment-short_name', 
                                foriegn_table='deployment', 
                                foriegn_column='short_name')
errors

2,short_name,foreign-deployment-short_name,suggestions
11,iphex_se_1,IPHEx_dep_2014,[GCPEx_dep_2012]
27,seac4rs_iop_1,SEAC4RS_dep_2013,[]
28,atom_iop_1,ATom_dep_2016,[]
29,atom_iop_2,ATom_dep_2017a,[]
30,atom_iop_3,ATom_dep_2017b,[]
...,...,...,...
240,tcsp_iop_1,TCSP_dep_2005,[]
241,tcsp_se_1,TCSP_dep_2005,[]
242,tcsp_se_2,TCSP_dep_2005,[]
243,tcsp_se_3,TCSP_dep_2005,[]


### Flight

In [86]:
# flight table, instruments

errors = validate.foriegn_keys(db, 
                               data_table='collection_period', 
                              data_index='foreign-campaign-short_name', 
                              data_column='instrument', 
                              foriegn_table='instrument', 
                              foriegn_column='short_name')
errors

,foreign-campaign-short_name,instrument,suggestions
81,GCPEx,NAWX radar,[]


In [87]:
# Flight table, platforms

errors = validate.foriegn_keys(db, 
                      data_table='collection_period', 
                      data_index='foreign-campaign-short_name', 
                      data_column='foreign-platform-short_name', 
                      foriegn_table='platform', 
                      foriegn_column='short_name')
errors

,foreign-campaign-short_name,foreign-platform-short_name,suggestions
19,OLYMPEX,UND Citation II,[Citation]
21,OLYMPEX,Field_Site,[]


In [88]:
# I think to ignore this???

errors = validate.foriegn_keys(db, 
                      data_table='collection_period', 
                      data_index='foreign-campaign-short_name', 
                      data_column='short_name',#'foreign-deployment-short_name', 
                      foriegn_table='deployment', 
                      foriegn_column='short_name')
errors

,foreign-campaign-short_name,short_name,suggestions
17,OLYMPEX,OLYMPEX_dep_2016,[OLYMPEX_dep_2015]
18,OLYMPEX,OLYMPEX_dep_2016,[OLYMPEX_dep_2015]
19,OLYMPEX,OLYMPEX_dep_2016,[OLYMPEX_dep_2015]
20,OLYMPEX,OLYMPEX_dep_2016,[OLYMPEX_dep_2015]
21,OLYMPEX,OLYMPEX_dep_2016,[OLYMPEX_dep_2015]


# Dates

In [89]:
for table_name in db.keys():
    # find tables that have date fields
    if len(col_names := [col for col in db[table_name].columns if 'date' in col])>0:
        print(f"{table_name}\n    {', '.join(col_names)}")

campaign
    start_date, end_date, ignore_metadata_date
deployment
    start_date, end_date
iopse
    start_date, end_date
iop
    start_date, end_date
significant_event
    start_date, end_date


In [90]:
table_names = ['campaign', 'deployment', 'iopse'] 
for table_name in table_names:
    print(table_name)
    db[table_name]['valid_date']=False
    
    db[table_name]['valid_date'] = db[table_name].apply(lambda row: validate.vali_date(row['start_date'], row['end_date']), axis=1)
        

campaign
deployment
iopse


In [ ]:
# this field no longer exists?

# table_names = ['instrument'] 
# for table_name in table_names:
    
#     db[table_name]['valid_date']=False
#     db[table_name]['valid_date'] = db[table_name].apply(lambda row: validate.vali_date(row['deployment_date'], row['decommision_date']), axis=1)
       

In [ ]:
db['campaign'][['short_name','start_date','end_date','valid_date']][db['campaign']['valid_date'].apply(lambda x: not(x))]

# IOPSE

In [ ]:
db['iop']

In [ ]:
# validate that all iops are unique, should return an empty list

db['iop']['short_name'][db['iop']['short_name'].duplicated()]

In [ ]:
# if sig event has matching iop, link them

# if sig event doesn't have matching iop, delete it so there will be no foriegn key

In [ ]:
# tag significant events that have IOPs

db['significant_event']['has_iop'] = db['significant_event']['iop_short_name'].apply(lambda short_name: short_name in list(db['iop']['iop_short_name']))


In [ ]:
# # check that all significant events with an iop have an iop

# s_i = set(db['iop']['iop_short_name'])
# s_s = set(has_iop['iop_short_name'])
# [s for s in s_s if s not in s_i]

In [ ]:
from datetime import datetime
# sig event start >= iop start

def start_val(sig_row, db):
    large_date = sig_row['start_date']
    small_date = db['iop'][db['iop']['iop_short_name']==sig_row['iop_short_name']]['start_date'].iloc[0]
    val = large_date >= small_date
    return val

def end_val(sig_row, db):
    large_date = sig_row['end_date']
    small_date = db['iop'][db['iop']['iop_short_name']==sig_row['iop_short_name']]['end_date'].iloc[0]
    val = large_date <= small_date
    return val

has_iop = db['significant_event'][db['significant_event']['has_iop']].copy()
val_iop_date_start = has_iop.apply(lambda row: not(start_val(row, db)), axis=1)
val_iop_date_end = has_iop.apply(lambda row: not(end_val(row, db)), axis=1)

In [ ]:
# display incorrect start dates
has_iop[val_iop_date_start]

In [ ]:
# display incorrect end dates
has_iop[val_iop_date_end]

### Pickle the Data

In [52]:
import pickle

In [53]:
pickle.dump(db, open('post_ingest','wb'))

In [ ]:
db['geographical_region']

In [ ]:
db['significant_event']